# Physics-Informed-Neural-Network to solve Partial Differential Equation
- Aim: To compare performance with Finite-Volume Implicit Numerical Scheme

## Problems with traditional methods: -
- High computational costs
- Boundary problems
- Computational costs for mesh generation
- Scaling for complex systems
- Numerical Instability

## Advantages of Neural Networks: -
- Adaptive learning- hence, may approximate function more efficiently
- Auto Differentiation- Key aspect of NN (gradient descent) that can also be applied to PDE solving for precise derivative calculations more efficienty
- PINNs use a cumulative loss function that ensures the computed answer complies with both- the data and the laws of physics that govern the PDE 
- Hence, appear to be more accurate approximations by embedding physics
- Consequently, should be more data efficient

## Risks to explore: -
- Training complexity
- Overfitting
- Numerical Stability

In [31]:
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

import keras_tuner as kt

import numpy as np

# Data type
DTYPE='float32'
tf.keras.backend.set_floatx(DTYPE)

## Mathematical Setup: -
### Idea: -

We construct a Neural Network approximation,
$$
u_\theta(t, x) \approx u(t,x),
$$

where $u_\theta : [0, T] \times \mathcal{D} \to \mathbb{R}$ denotes a function approximated by a neural network with parameters $\theta$.

### Example: Burger's Equation

$$
\begin{aligned}
    \partial_t u + u.\partial_x u - v.\partial_{xx} u &= 0, \quad && \quad (t, x) \in (0, 1] \times (-1, 1);\\
   u(0, x) &= -\sin(\pi.x), \quad && \quad x \in [-1,1];\\
   u(t, -1) &= u(t, 1) = 0, \quad && \quad t \in (0,1];
\end{aligned}
$$

In [32]:
# Constants
v = tf.constant(1, dtype=tf.float32)  
pi = tf.constant(np.pi, dtype=tf.float32)

# Initial condition
def u_0(x):
    return -tf.sin(pi*x)    # u_0(x) = sin(pi.x)

# Flux
def f(u_xx):
    return v*u_xx

# PDE residual using TensorFlow AutoDiff
def pde_residual(model, t, x):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([t, x])
        u = model(tf.concat([t, x], axis=1))
        u_t = tape.gradient(u, t)
        u_x = tape.gradient(u, x)
    u_xx = tape.gradient(u_x, x)
    del tape

    # Burgers' equation: u_t + u * u_x = v * u_xx
    return u_t + u*u_x - f(u_xx)

## Network Architecture

In [33]:
def build_network(hp):
    # Hyperparameters
    units = hp.Int('units', min_value=2, max_value=128, step=2)
    layers = hp.Int('layers', min_value=1, max_value=64)
    activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])
    
    # Model architecture
    input_layer = Input(shape=(2,))
    x = input_layer
    for _ in range(layers):
        x = Dense(units, activation=activation, kernel_initializer = 'glorot_normal')(x)
    output_layer = Dense(1, activation=None, kernel_initializer = 'glorot_normal')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(5e-4), loss='mse')
    
    return model

# Loss functions
# Enforce physical lalws (PDE and boundary conditions)
@tf.function
def pinn_loss(model, t, x):
    f = pde_residual(model, t, x)
    #Implement boundary condition loss
    return tf.reduce_mean(tf.square(f))

# Data Loss
@tf.function
def model_loss(model, t, x):
    dif = model(tf.concat([t, x], axis=1)) - u_0(x)
    return tf.reduce_mean(tf.square(dif))


## Training

In [34]:
# Custom training step
epochs = 256

def train_model(model, t_train, x_train):
    optimizer = tf.keras.optimizers.Adam(0.001)
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss = pinn_loss(model, t_train, x_train) + model_loss(model, t_train, x_train)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        if epoch % 8 == 0:
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

### Training Data

In [35]:
# Training data
def generate_data(num_x, num_t, xmin, xmax, tmin, tmax):
    t = tf.convert_to_tensor(np.linspace(tmin, tmax, num_t).astype(np.float32).reshape(-1, 1), dtype=tf.float32)
    x = tf.convert_to_tensor(np.linspace(xmin, xmax, num_x).astype(np.float32).reshape(-1, 1), dtype=tf.float32)
    U = u_0(x)  

    return U, x, t

# Parameters
x_points = 512
t_points = 512
xmin, xmax = -1, 1
tmin, tmax = 0, 1

U, x_train, t_train = generate_data(x_points, t_points, xmin, xmax, tmin, tmax)
input_data = np.hstack((t_train, x_train))

### Hyperparameter Tuning

In [36]:
def trainer(hp):
    model = build_network(hp)
    train_model(model, t_train, x_train)
    
    return model
    
# Remove last config
import os
os.system(r'"rm -r D:\Programming\proj\PINN_PDE\pinn_tuning"')  

# Tuner
tuner = kt.BayesianOptimization(
    trainer,
    objective = 'loss',
    max_trials = 16,
    max_retries_per_trial = 1,
    directory = 'pinn_tuning',
    project_name = 'burgers'
)

# Callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 5)

# Search
tuner.search(
    x = input_data,
    epochs = 64,
    validation_split = 0.2,
    callbacks = [early_stopping]
); best_model = tuner.get_best_models(num_models=1)[0]


Trial 3 Complete [00h 00m 12s]

Best loss So Far: None
Total elapsed time: 00h 03m 14s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
20                |46                |units
9                 |33                |layers
relu              |sigmoid           |activation

Epoch 0, Loss: 0.5012192726135254
Epoch 8, Loss: 0.47349128127098083
Epoch 16, Loss: 0.42952650785446167
Epoch 24, Loss: 0.3541991114616394
Epoch 32, Loss: 0.24233485758304596
Epoch 40, Loss: 0.17840388417243958
Epoch 48, Loss: 0.18253451585769653
Epoch 56, Loss: 0.194713294506073
Epoch 64, Loss: 0.21540483832359314
Epoch 72, Loss: 0.2124156653881073
Epoch 80, Loss: 0.19158582389354706
Epoch 88, Loss: 0.1887846291065216
Epoch 96, Loss: 0.18705442547798157
Epoch 104, Loss: 0.1855792999267578
Epoch 112, Loss: 0.1849203109741211
Epoch 120, Loss: 0.18442505598068237
Epoch 128, Loss: 0.18413817882537842
Epoch 136, Loss: 0.18679682910442352
Epoch 144, Loss: 0.18958213925361633
Epoch 152, Los

Traceback (most recent call last):
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^^^^^^^

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.

In [ ]:
best_model.summary()

In [ ]:
# Best Model
train_model(best_model, t_train, x_train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 4), dpi=300)

# Number of temporal and spatial points
n_temporal, n_spatial = 10, 2000

# Combine (x, t) as a vector
v = np.zeros([n_spatial, 2])  # v[:, 0] = t, v[:, 1] = x
v[:, 1] = np.linspace(-1, +1, n_spatial)  # Spatial coordinate

# Change t for different lines
for i in range(n_temporal):
    v[:, 0] = i / n_temporal  # Temporal coordinate
    u_pred = best_model.predict(v)  # Predict u(t, x)
    plt.plot(v[:, 1], u_pred, label=f"t = {i / n_temporal:.2f}", lw=0.75)

plt.legend(loc="upper right", fontsize=5)
plt.xlim(-1, +1)
plt.ylim(-1, +1)
plt.ylabel("u(t, x)")
plt.xlabel("x")
plt.tight_layout()
plt.show()

In [ ]:
best_model.predict(input_data)

## Model Usage

### Simulation Setup

In [ ]:

# # Simulation Parameters

# # Number of points
# N_0 = 64
# N_b = 64
# N_r = 8320


# # Mesh
# tmin, tmax = 0., 1.
# xmin, xmax = -1., 1.


# # Lower bounds
# lb = tf.constant([tmin, xmin], dtype=DTYPE)
# # Upper bounds
# ub = tf.constant([tmax, xmax], dtype=DTYPE)


# # Draw uniform sample points for initial boundary data
# tf.random.set_seed(123)     #seed for reproducible results
# t_0 = tf.ones((N_0,1), dtype=DTYPE)*lb[0]
# x_0 = tf.random.uniform((N_0,1), lb[1], ub[1], dtype=DTYPE)
# X_0 = tf.concat([t_0, x_0], axis=1)


# # Boundary data
# t_b = tf.random.uniform((N_b,1), lb[0], ub[0], dtype=DTYPE)
# x_b = lb[1] + (ub[1] - lb[1]) * tf.keras.backend.random_bernoulli((N_b,1), 0.5, dtype=DTYPE)
# X_b = tf.concat([t_b, x_b], axis=1)

# X_data = [X_0, X_b]

# # Draw uniformly sampled collocation points
# t_r = tf.random.uniform((N_r,1), lb[0], ub[0], dtype=DTYPE)
# x_r = tf.random.uniform((N_r,1), lb[1], ub[1], dtype=DTYPE)
# X_r = tf.concat([t_r, x_r], axis=1)

# # u_0 = fun_u_0(x_0)          # Initial data
# # u_b = fun_u_b(t_b, x_b)     # Boundary data

# # u_data = [u_0, u_b]

## Mesh
- collocation points: red circles
- boundary and initial conditions: cross marks (colour indicates value)

In [ ]:
# # Evaluate the model
# u_pred = best_model(tf.concat([t_train, x_train], axis=1))
# print("Final loss:", pinn_loss(best_model, t_train, x_train, t_bc, x_bc, u_bc).numpy(), u_pred)

In [4]:
import matplotlib.pyplot as plt

plt.figure(dpi = 5000)
x = 1
y = 2
plt.plot(x,y)